In [51]:
%%writefile .env
YELP_API_KEY=iuOGUxONHPfHGy4m6zATyltGnwzJkbLmGeOZ_fiH_0nY7dSbcLS72YfSKnBjfh5H-T3WtwUdFLCSe_RIUORYv6ELJVQEH_JhozgaMye1-yFSFq-yFcIc9cd4VGHZaHYx

Overwriting .env


In [ ]:
city_name_caps = "Dallas, TX"
city_name_lower = "dallas"

SyntaxError: invalid syntax (4120329305.py, line 1)

I have used my API key above. Please use yours respectively.

In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
# load variables from .env
load_dotenv()
# Fetch your Yelp API key from the environment
YELP_API_KEY = os.getenv("YELP_API_KEY")
# Check it loaded correctly (show only first 6 characters)
print("API Key loaded:", YELP_API_KEY[:6], "....")

In [ ]:
# Step 0: Install dependencies
#!pip install python-dotenv pandas requests

# Step 1: Load API key from .env
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import time

load_dotenv()
YELP_API_KEY = os.getenv("YELP_API_KEY")
print("Yelp API Key loaded:", YELP_API_KEY[:6], "....")

HEADERS = {"Authorization": f"Bearer {YELP_API_KEY}"}
SEARCH_URL = "https://api.yelp.com/v3/businesses/search"
DETAILS_URL = "https://api.yelp.com/v3/businesses/{}"
REVIEWS_URL = "https://api.yelp.com/v3/businesses/{}/reviews"

# Step 2: Function to fetch restaurants with offset limits # Change location appropriately
def fetch_restaurants(location=city_name_caps, term="restaurants", max_fetch=200):
    results = []
    limit = 50  # Yelp max per request
    for offset in range(0, max_fetch, limit):
        params = {"location": location, "term": term, "limit": limit, "offset": offset}
        r = requests.get(SEARCH_URL, headers=HEADERS, params=params)
        if r.status_code != 200:
            print(f"Error {r.status_code}: {r.text}")
            break
        chunk = r.json().get("businesses", [])
        if not chunk:
            break
        results.extend(chunk)
        print(f"Fetched {len(results)} for term '{term}' so far...")
        time.sleep(0.5)
    return results

# Step 3: Categories to split queries
categories = ["italian", "mexican", "indian", "chinese", "thai", "american", "mediterranean", "vegan", "vegetarian"]

all_restaurants = []

for cat in categories:
    results = fetch_restaurants(term=f"restaurants,{cat}", max_fetch=200)
    all_restaurants.extend(results)

# Deduplicate by restaurant id
unique_restaurants = {biz['id']: biz for biz in all_restaurants}
restaurants = list(unique_restaurants.values())
print(f"Total unique restaurants fetched: {len(restaurants)}")


In [ ]:
# Step 4: Fetch business details + reviews
all_data = []
sample_restaurants = restaurants
for i, biz in enumerate(sample_restaurants):
    biz_id = biz['id']

    # Details
    details_resp = requests.get(DETAILS_URL.format(biz_id), headers=HEADERS)
    if details_resp.status_code != 200:
        print(f"Details error for {biz_id}")
        continue
    details = details_resp.json()

    # Try to capture website / menu links
    website_url = details.get("url", "")  # Yelp page
    try:
        website_url = details.get("attributes", {}).get("menu_url", website_url)  # Official menu if available
    except:
        pass

    # Save combined data
    all_data.append({
        "id": biz_id,
        "name": details.get("name"),
        "rating": details.get("rating"),
        "review_count": details.get("review_count"),
        "categories": ", ".join([c['title'] for c in details.get("categories", [])]),
        "price": details.get("price", ""),
        "address": ", ".join(details["location"].get("display_address", [])),
        "latitude": details["coordinates"].get("latitude"),
        "longitude": details["coordinates"].get("longitude"),
        "url": details.get("url"),
        "website_url": website_url  # <-- NEW FIELD
    })

    if (i+1) % 50 == 0:
        print(f"Processed {i+1}/{len(restaurants)} restaurants")
    time.sleep(0.5)


In [ ]:
# Step 5: Save to CSV
df = pd.DataFrame(all_data)
df["city"] = city_name_caps
df.to_csv(f"{city_name_lower}_restaurants_full_with_websites.csv", index=False)
print(f"Saved to {city_name_lower}_restaurants_full_with_websites.csv")


The above code will generate a dataset containing all the restaurant details for the city of San Diego. Please change the code to update the location to whatever city you are looking at.

In [ ]:
#!pip install pdfplumber requests beautifulsoup4 pandas
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io
import pdfplumber
#from google.colab import drive

# === Function to extract text from PDF ===
def get_pdf_text(pdf_bytes):
    """
    Extracts text from a PDF file provided as bytes.
    """
    full_text = []
    try:
        with pdfplumber.open(io.BytesIO(pdf_bytes)) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    full_text.append(text)
        return "\n".join(full_text) if full_text else "ERROR: No text extracted from PDF"
    except Exception as e:
        return f"ERROR extracting PDF: {e}"

# === Function to fetch website text (HTML or PDF) ===
def get_website_text(url):
    """
    Fetches the content of a website (HTML or PDF) and returns the plain text.
    """
    try:
        if not isinstance(url, str) or url.strip() == "":
            return "ERROR: No URL"

        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/115.0 Safari/537.36"
            )
        }

        print(f"Fetching content from: {url}")
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()

        content_type = response.headers.get("Content-Type", "").lower()

        if "application/pdf" in content_type or url.lower().endswith(".pdf"):
            print("📄 Detected PDF, extracting text...")
            return get_pdf_text(response.content)
        else:
            print("🌐 Detected HTML, extracting text...")
            soup = BeautifulSoup(response.text, "html.parser")
            text = soup.get_text(separator="\n", strip=True)
            return text if text else "ERROR: No text extracted from HTML"

    except Exception as e:
        return f"ERROR: {e}"

# === Load dataset === Upload the dataset you got from the above code to the same folder where you are running your google colab notebook.

#drive.mount('/content/drive')
df = pd.read_csv(f"{city_name_lower}_restaurants_full_with_websites.csv")


# Test on first 10 rows before running everything
df_test = df
df_test["raw_text"] = df_test["website_url"].apply(get_website_text)

# Save the test run
df_test.to_csv(f"{city_name_lower}_restaurants_test_with_rawtext.csv", index=False)
print(f"✅ Test file saved: {city_name_lower}_restaurants_test_with_rawtext.csv")

# Once verified, uncomment to run full dataset:
# df["raw_text"] = df["website_url"].apply(get_website_text)
# df.to_csv("san_diego_restaurants_full_with_rawtext.csv", index=False)
# print("✅ Full dataset saved with raw_text")


The above code will generate a dataset with the raw text added as a column

In [ ]:
# The code below is used to calculate how many entries have null/empty strings/error in extracting raw text
import pandas as pd

# Load dataset
#drive.mount('/content/drive')
df = pd.read_csv(f"{city_name_lower}_restaurants_test_with_rawtext.csv")

# Define "error" conditions
mask_error = (
    df["raw_text"].isna() |  # missing values (NaN)
    (df["raw_text"].astype(str).str.contains("ERROR", case=False, na=False)) |  # text contains 'error'
    (df["raw_text"].astype(str).str.strip().isin(["", "None", "nan"]))  # empty, None, nan as strings
)

# Count errors
error_count = mask_error.sum()

print(f"Number of entries with errors in raw_text: {error_count}")
print(f"Total entries: {len(df)}")
print(f"Percentage errors: {100 * error_count / len(df):.2f}%")